In [1]:
import os

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from delta.tables import DeltaTable
from delta import configure_spark_with_delta_pip
import shutil

# this Clears any previous runs
shutil.rmtree("/tmp/delta-table", ignore_errors=True)

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1,' + 'io.delta:delta-core_2.12:2.2.0,' + 'com.amazonaws:aws-java-sdk-s3:1.11.1026') # basically most recent version as my spark version
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set('spark.hadoop.fs.s3a.access.key', os.environ.get('aws_access_key_id'))
conf.set('spark.hadoop.fs.s3a.secret.key', os.environ.get('aws_secret_access_key'))
conf.set('spark.sql.extensions', "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

builder = SparkSession.builder.appName("delta_praq") \
   .config(conf = conf)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/03/16 16:11:37 WARN Utils: Your hostname, jacob-BigOtisLinux resolves to a loopback address: 127.0.1.1; using 192.168.50.142 instead (on interface enp6s0)
23/03/16 16:11:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/jacob/.local/share/virtualenvs/pyspark_prac-QhYfwHaC/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jacob/.ivy2/cache
The jars for the packages stored in: /home/jacob/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f6832ff6-ac11-4c68-836e-f1475e5803e6;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 83ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

23/03/16 16:11:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
print("############# Creating a table ###############")
data = spark.range(0, 5)
# data.write.format("delta").save("/tmp/delta-table", mode = 'OVERWRITE')



############# Creating a table ###############


In [3]:
deltaTable = DeltaTable.forPath(spark, '/tmp/delta-table')

fullHistoryDF = deltaTable.history()    # get the full history of the table

lastOperationDF = deltaTable.history(1) # get the last operation

In [6]:
fullHistoryDF.toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,0,2022-08-07 10:51:49.383,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '5', 'numOutputBytes': '2686...",None,Apache-Spark/3.2.2 Delta-Lake/2.0.0


In [7]:
lastOperationDF.toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,0,2022-08-07 10:51:49.383,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '5', 'numOutputBytes': '2686...",None,Apache-Spark/3.2.2 Delta-Lake/2.0.0


In [3]:
data.write.format("delta").save("s3a://jyablonski-delta/spark_delta/delta_table1.delta")

Py4JJavaError: An error occurred while calling o43.save.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:726)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:598)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:151)
	at org.apache.spark.sql.execut